In [ ]:
%matplotlib inline
# Import Dependencies
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

# Read the CSV into a Pandas DataFrame
intake_csv = "Resources/Austin_Animal_Center_Intakes.csv"
outcome_csv= "Resources/Austin_Animal_Center_Outcomes.csv"

intake_df = pd.read_csv(intake_csv)
outcome_df = pd.read_csv(outcome_csv)

#convert date variable to datetime format

intake_df['DateTime'] = pd.to_datetime(intake_df['DateTime']).dt.date
outcome_df['DateTime'] = pd.to_datetime(outcome_df['DateTime']).dt.date

#update intake file with column reflecting number of intakes
#and select outcomes for animals with 1 or 2 intakes
intake_counts=intake_df.groupby(['Animal ID']).size().reset_index(name='counts')
intake_df=pd.merge(intake_df, intake_counts, on="Animal ID", how="outer")
reduced_intake_df=intake_df.query('counts <3')

#update outtake file with column reflecting number of intakes per animal
#and select outcomes for animals with 1 or 2 intakes
outcome_df=pd.merge(outcome_df, intake_counts, on="Animal ID", how="left")
reduced_outcome_df=outcome_df.query('counts <3')
#reduced_outcome_df=reduced_outcome_df.drop(columns=['counts'])

In [ ]:
outcome_df

In [ ]:
reduced_outcome_df

In [ ]:
#now take single intake animals and match with any available outcome
reduced_intake_only_one_df=reduced_intake_df.query('counts <2')
#reduced_intake_only_one_df=reduced_intake_only_one_df.add_prefix('IN_')
reduced_outcome_df=reduced_outcome_df.add_prefix("OUT_")

reduced_intake_only_one_df=reduced_intake_only_one_df.rename(index=str,columns={"IN_Animal ID": "Animal ID"})
reduced_outcome_df=reduced_outcome_df.rename(index=str,columns={"OUT_Animal ID": "Animal ID"})
one_intake_merged=pd.merge(reduced_intake_only_one_df, reduced_outcome_df, on="Animal ID", how="left")

In [ ]:
one_intake_merged["num_of_intakes_for_ID"]=1

In [ ]:
#now take two intake animals and figure out order of intakes and outcomes
reduced_intake_two_df=reduced_intake_df.query('counts !=1')


In [ ]:
reduced_intake_two_first_date_lookup_df=reduced_intake_two_df.groupby('Animal ID').aggregate(['min']).reset_index()
reduced_intake_two_first_date_lookup_df = reduced_intake_two_first_date_lookup_df[['Animal ID','DateTime']]
reduced_intake_two_first_date_lookup_df["stay_number"]=1
reduced_intake_two_first_date_lookup_df.columns = reduced_intake_two_first_date_lookup_df.columns.droplevel(1)


reduced_intake_two_first_date_lookup_df.head()

In [ ]:
reduced_intake_two_second_date_lookup_df=reduced_intake_two_df.groupby('Animal ID').aggregate(['max']).reset_index()
reduced_intake_two_second_date_lookup_df = reduced_intake_two_second_date_lookup_df[['Animal ID','DateTime']]
reduced_intake_two_second_date_lookup_df["stay_number"]=2
reduced_intake_two_second_date_lookup_df.columns = reduced_intake_two_second_date_lookup_df.columns.droplevel(1)

In [ ]:
reduced_intake_two_second_date_lookup_df.head()

In [ ]:
multiple_intakes_order_lookup = reduced_intake_two_first_date_lookup_df.append(reduced_intake_two_second_date_lookup_df)

In [ ]:
#multiple_intakes_order_lookup

In [ ]:
#reduced_intake_two_df

reduced_intake_two_df=pd.merge(reduced_intake_two_df, multiple_intakes_order_lookup, on=["Animal ID","DateTime"], how="left")
#reduced_intake_two_df

In [ ]:
#look at outcome records for the two-intake group and determine order

In [ ]:
#reduced_outcome_df

In [ ]:

reduced_outcome_two_first_date_lookup_df=reduced_outcome_df.groupby('Animal ID').aggregate(['min']).reset_index()
reduced_outcome_two_first_date_lookup_df = reduced_outcome_two_first_date_lookup_df[['Animal ID','OUT_DateTime']]
reduced_outcome_two_first_date_lookup_df["stay_number"]=1
reduced_outcome_two_first_date_lookup_df.columns = reduced_outcome_two_first_date_lookup_df.columns.droplevel(1)


reduced_outcome_two_first_date_lookup_df.head()

In [ ]:
reduced_outcome_df=pd.merge(reduced_outcome_df, reduced_outcome_two_first_date_lookup_df, on=["Animal ID","OUT_DateTime"], how="left")

reduced_outcome_df

In [ ]:
reduced_outcome_df['stay_number']=reduced_outcome_df['stay_number'].fillna(2)
#reduced_outcome_df

In [ ]:
#reduced_intake_two_df=reduced_intake_two_df.add_prefix('IN_')
reduced_intake_two_df=reduced_intake_two_df.rename(index=str,columns={"IN_Animal ID": "Animal ID","IN_stay_number":"stay_number"})
reduced_intake_two_df

In [ ]:
#merge the multiple intake with coressponsing outcomes - will not yield two outcomes for animals still in residence
two_intakes_merged=pd.merge(reduced_intake_two_df, reduced_outcome_df, on=["Animal ID", "stay_number"], how="left")
two_intakes_merged["num_of_intakes_for_ID"]="2"

In [ ]:
two_intakes_merged

In [ ]:
one_intake_merged

In [ ]:
working_base_master=one_intake_merged.append(two_intakes_merged, ignore_index=True)

In [ ]:
working_base_master.head()

In [ ]:
working_base_master['time_to_exit'] = working_base_master['OUT_DateTime']-working_base_master['DateTime']

In [ ]:
working_base_master['time_to_exit']=working_base_master['time_to_exit'] / np.timedelta64(1, 'D')

In [ ]:
working_base_master['time_to_exit'].dtype

In [ ]:
working_base_master.head()

In [ ]:
############################

In [ ]:
#Discover maximum time_to_exit
working_base_master["time_to_exit"].max()

In [ ]:
#Remove all time_to_exit negative values
test_df = working_base_master[working_base_master['time_to_exit'].between(0, 2000)]
test_df

In [ ]:
#filter to only include Owner Surrender and Stray
test_df = test_df[test_df['Intake Type'].isin(["Owner Surrender", "Stray"])]
test_df

In [ ]:
#filter to only normal intake type
test_df = test_df.loc[test_df["Intake Condition"] == "Normal", :]

test_df.head()

In [ ]:
#Create dataframe of named pets at outcome by dropping no names
name_df = test_df[test_df['OUT_Name'].notnull()]
name_df

In [ ]:
#Display value counts for dataframe
name_df["OUT_Outcome Type"].value_counts()

In [ ]:
#Confirm NaN values removed
name_df["OUT_Name"].value_counts()

In [ ]:
#filter out outcome types besides adoption, transfer or Return to Owner
name_df = name_df[name_df['OUT_Outcome Type'].isin(["Adoption", "Return to Owner", "Transfer"])]
name_df

In [ ]:
#Confirm all outcome types removed besides adoption, transfer or Return to Owner
name_df["OUT_Outcome Type"].value_counts()

In [ ]:
#Fill outcome subtypes with NaN to None
name_df = name_df = name_df.fillna({'OUT_Outcome Subtype':'None'})

In [ ]:
#See what adoption subtypes remain
name_df["OUT_Outcome Subtype"].value_counts()

In [ ]:
#Filter data subtypes to remove foster and barn
name_df = name_df[name_df['OUT_Outcome Subtype'].isin(["None", "Partner", "SCRP", "Snr"])]
name_df

In [ ]:
#See what adoption subtypes remain
name_df["OUT_Outcome Subtype"].value_counts()

In [ ]:
#See what adoption types remain
name_df["OUT_Outcome Type"].value_counts()

In [ ]:
#total outcome types (named)
name_all_count = name_df["OUT_Outcome Type"].count()
name_all_count

In [ ]:
name_adoption = (name_df.loc[name_df["OUT_Outcome Type"] == "Adoption", :])

In [ ]:
#count of adopted (named)
name_adoption_count = name_adoption["OUT_Outcome Type"].count()
name_adoption_count

In [ ]:
name_transfer = (name_df.loc[name_df["OUT_Outcome Type"] == "Transfer", :])

In [ ]:
#count of transfered (named)
name_transfer_count = name_transfer["OUT_Outcome Type"].count()
name_transfer_count

In [ ]:
name_return_to_owner = (name_df.loc[name_df["OUT_Outcome Type"] == "Return to Owner", :])

In [ ]:
#count of returned to owner (named)
name_return_to_owner_count = name_return_to_owner["OUT_Outcome Type"].count()
name_return_to_owner_count

In [ ]:
name_pct_adopted = name_adoption_count/name_all_count
name_pct_adopted

In [ ]:
name_per_capita_adopted = name_pct_adopted * 100
name_per_capita_adopted

In [ ]:
name_pct_transfer = name_transfer_count/name_all_count
name_pct_transfer

In [ ]:
name_per_capita_transfer = name_pct_transfer * 100
name_per_capita_transfer

In [ ]:
name_pct_return_to_owner = name_return_to_owner_count/name_all_count
name_pct_return_to_owner

In [ ]:
name_per_capita_return_to_owner = name_pct_return_to_owner * 100
name_per_capita_return_to_owner

In [ ]:
######################

In [ ]:
test_df.head()

In [ ]:
no_name_df = test_df.fillna({'OUT_Name':'None'})

In [ ]:
#Create dataframe of unnamed pets at outcome by filtering to just none
no_name_df = no_name_df.loc[no_name_df["OUT_Name"] == "None", :]

no_name_df.head()

In [ ]:
no_name_df["OUT_Outcome Type"].value_counts()

In [ ]:
#Confirm NaN values removed
no_name_df["OUT_Name"].value_counts()

In [ ]:
#filter out outcome types besides adoption, transfer or Return to Owner
no_name_df = no_name_df[no_name_df['OUT_Outcome Type'].isin(["Adoption", "Return to Owner", "Transfer"])]
no_name_df

In [ ]:
#Confirm all outcome types removed besides adoption, transfer or Return to Owner
no_name_df["OUT_Outcome Type"].value_counts()

In [ ]:
no_name_df = no_name_df = no_name_df.fillna({'OUT_Outcome Subtype':'None'})

In [ ]:
#See what adoption subtypes remain
no_name_df["OUT_Outcome Subtype"].value_counts()

In [ ]:
no_name_df = no_name_df[no_name_df['OUT_Outcome Subtype'].isin(["None", "Partner", "SCRP", "Snr"])]
no_name_df

In [ ]:
no_name_df["OUT_Outcome Subtype"].value_counts()

In [ ]:
no_name_df["OUT_Outcome Type"].value_counts()

In [ ]:
no_name_all_count = no_name_df["OUT_Outcome Type"].count()
no_name_all_count

In [ ]:
no_name_adoption = (no_name_df.loc[no_name_df["OUT_Outcome Type"] == "Adoption", :])

In [ ]:
no_name_adoption_count = no_name_adoption["OUT_Outcome Type"].count()
name_adoption_count

In [ ]:
no_name_transfer = (no_name_df.loc[no_name_df["OUT_Outcome Type"] == "Transfer", :])

In [ ]:
no_name_transfer_count = no_name_transfer["OUT_Outcome Type"].count()
no_name_transfer_count

In [ ]:
no_name_return_to_owner = (no_name_df.loc[no_name_df["OUT_Outcome Type"] == "Return to Owner", :])

In [ ]:
no_name_return_to_owner_count = no_name_return_to_owner["OUT_Outcome Type"].count()
no_name_return_to_owner_count

In [ ]:
no_name_pct_adopted = no_name_adoption_count/no_name_all_count
no_name_pct_adopted

In [ ]:
no_name_per_capita_adopted = no_name_pct_adopted * 100
no_name_per_capita_adopted

In [ ]:
no_name_pct_transfer = no_name_transfer_count/no_name_all_count
no_name_pct_transfer

In [ ]:
no_name_per_capita_transfer = no_name_pct_transfer * 100
no_name_per_capita_transfer

In [ ]:
no_name_pct_return_to_owner = no_name_return_to_owner_count/no_name_all_count
no_name_pct_return_to_owner

In [ ]:
no_name_per_capita_return_to_owner = no_name_pct_return_to_owner * 100
no_name_per_capita_return_to_owner

In [ ]:
N = 3

no_name_chart = [no_name_per_capita_transfer, no_name_per_capita_adopted, no_name_per_capita_return_to_owner]
name_chart = [name_per_capita_transfer, name_per_capita_adopted, name_per_capita_return_to_owner]


fig, ax = plt.subplots()

ind = np.arange(N)    # the x locations for the groups
width = 0.40         # the width of the bars
p1 = ax.bar(ind, no_name_chart, width, color='#217fd1')

p2 = ax.bar(ind + width, name_chart, width, color='#21d1b3')

ax.set_title('Outcomes Per 100 Animals', fontweight='bold')
ax.set_xticks(ind + width / 2)
ax.set_xticklabels(('Transfered', 'Adopted', 'Returned To Owner'))

ax.set_ylabel('Number of Occurances', fontweight='bold')
ax.set_xlabel('Outcomes', fontweight='bold')

ax.legend((p1[0], p2[0]), ('Unnamed', 'Named'))
ax.autoscale_view()
ax.grid(axis='y', alpha=.20)
plt.savefig("Outcomes Name Vs. No Name.png", dpi=1020)
plt.show()

In [ ]:
# set width of bar
barWidth = 0.25
 
# set height of bar
transfered_chart = [no_name_per_capita_transfer, name_per_capita_transfer]
adopted_chart = [no_name_per_capita_adopted, name_per_capita_adopted]
return_to_owner_chart = [no_name_per_capita_return_to_owner, name_per_capita_return_to_owner]
 
# Set position of bar on X axis
r1 = np.arange(len(transfered_chart))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
 
# Make the plot
plt.bar(r1, transfered_chart, color='#217fd1', width=barWidth, edgecolor='white', label='Transfered')
plt.bar(r2, adopted_chart, color='#ed9b04', width=barWidth, edgecolor='white', label='Adopted')
plt.bar(r3, return_to_owner_chart, color='#21d1b3', width=barWidth, edgecolor='white', label='Returned To Owner')

#21d1b3

# Add xticks on the middle of the group bars
plt.xlabel('Outcomes', fontweight='bold')
plt.xticks([r + barWidth for r in range(len(transfered_chart))], ['Unnamed', 'Named'])
plt.ylabel('Number of Occurances', fontweight='bold')
plt.title('Outcomes Per 100 Animals', fontweight='bold')
# Create legend & Show graphic
plt.legend()
plt.grid(axis='y', alpha=.20)
plt.savefig("Outcomes Unnamed Vs. Named.png", dpi=1020)
plt.show()

In [ ]:
######################################

In [ ]:
test_df.head()

In [ ]:
list(test_df)

In [ ]:
#test_df = test_df = test_df.fillna({'stay_number':'None'})

In [ ]:
test_df["stay_number"].count()

In [ ]:
test_df["stay_number"].value_counts()

In [ ]:
test_df["num_of_intakes_for_ID"].value_counts()

In [ ]:
working_base_master.head()

In [ ]:
return_df = working_base_master[working_base_master['time_to_exit'].between(0, 2000)]
return_df

In [ ]:
return_df = return_df[return_df['Intake Type'].isin(["Owner Surrender", "Stray"])]
return_df

In [ ]:
return_df = return_df.loc[return_df["Intake Condition"] == "Normal", :]

return_df.head()

In [ ]:
dog_return_df = return_df.loc[return_df["Animal Type"] == "Dog", :]

dog_return_df.head()

In [ ]:
dog_return_df = dog_return_df[dog_return_df['OUT_Outcome Type'].isin(["Adoption"])]
dog_return_df

In [ ]:
dog_return_df = dog_return_df.fillna({'OUT_Outcome Subtype':'None'})

In [ ]:
#See what adoption subtypes remain
dog_return_df["OUT_Outcome Subtype"].value_counts()

In [ ]:
dog_return_df = dog_return_df[dog_return_df['OUT_Outcome Subtype'].isin(["None"])]
dog_return_df

In [ ]:
pit_bull_total_adopted = dog_return_df[dog_return_df["OUT_Breed"].str.contains('Pit')]
pit_bull_total_adopted

In [ ]:
non_pit_bull_total_adopted = dog_return_df[dog_return_df["OUT_Breed"].str.contains('Pit') == False]
non_pit_bull_total_adopted

In [ ]:
#create 1 return df
#create 2 return df

#create 1 return dog df
#create 2 return dog df

#create 1 return pit bull df
#create 2 return non-pit bull df

#create 1 return pit bull df
#create 2 return non-pit bull df

In [ ]:
one_return_df = dog_return_df[dog_return_df['stay_number'].isin(["1.0"])]
one_return_df

In [ ]:
two_return_df = dog_return_df[dog_return_df['stay_number'].isin(["2.0"])]
two_return_df

In [ ]:
all_return_df = dog_return_df[dog_return_df['stay_number'].isin(["1.0", "2.0"])]
all_return_df

In [ ]:
one_return_df = one_return_df[one_return_df['Intake Type'].isin(["Owner Surrender"])]
one_return_df

In [ ]:
two_return_df = two_return_df[two_return_df['Intake Type'].isin(["Owner Surrender"])]
two_return_df

In [ ]:
all_return_df = all_return_df[all_return_df['Intake Type'].isin(["Owner Surrender"])]
all_return_df

In [ ]:
one_pit_bull_list = one_return_df[one_return_df['OUT_Breed'].str.contains('Pit')]
one_pit_bull_list.count()

In [ ]:
two_pit_bull_list = two_return_df[two_return_df['OUT_Breed'].str.contains('Pit')]

In [ ]:
all_pit_bull_list = all_return_df[all_return_df['OUT_Breed'].str.contains('Pit')]

In [ ]:
one_non_pit_bull_list = one_return_df[one_return_df['OUT_Breed'].str.contains('Pit') == False]

In [ ]:
two_non_pit_bull_list = two_return_df[two_return_df['OUT_Breed'].str.contains('Pit') == False]

In [ ]:
all_non_pit_bull_list = all_return_df[all_return_df['OUT_Breed'].str.contains('Pit') == False]

In [ ]:
one_pit_bull_return_pct = one_pit_bull_list['OUT_Breed'].count()/pit_bull_total_adopted['OUT_Breed'].count()
one_pit_bull_return_pct

In [ ]:
one_pit_bull_return_per_capita = one_pit_bull_return_pct * 100
one_pit_bull_return_per_capita

In [ ]:
two_pit_bull_return_pct = two_pit_bull_list['OUT_Breed'].count()/pit_bull_total_adopted['OUT_Breed'].count()
two_pit_bull_return_pct

In [ ]:
two_pit_bull_return_per_capita = two_pit_bull_return_pct * 100
two_pit_bull_return_per_capita

In [ ]:
all_pit_bull_return_pct = all_pit_bull_list['OUT_Breed'].count()/pit_bull_total_adopted['OUT_Breed'].count()
all_pit_bull_return_pct

In [ ]:
all_pit_bull_return_per_capita = all_pit_bull_return_pct * 100
all_pit_bull_return_per_capita

In [ ]:
one_non_pit_bull_return_pct = one_non_pit_bull_list['OUT_Breed'].count()/non_pit_bull_total_adopted['OUT_Breed'].count()
one_non_pit_bull_return_pct

In [ ]:
one_non_pit_bull_return_per_capita = one_non_pit_bull_return_pct * 100
one_non_pit_bull_return_per_capita

In [ ]:
two_non_pit_bull_return_pct = two_non_pit_bull_list['OUT_Breed'].count()/non_pit_bull_total_adopted['OUT_Breed'].count()
two_non_pit_bull_return_pct

In [ ]:
two_non_pit_bull_return_per_capita = two_non_pit_bull_return_pct * 100
two_non_pit_bull_return_per_capita

In [ ]:
all_non_pit_bull_return_pct = all_non_pit_bull_list['OUT_Breed'].count()/non_pit_bull_total_adopted['OUT_Breed'].count()
all_non_pit_bull_return_pct

In [ ]:
all_non_pit_bull_return_per_capita = all_non_pit_bull_return_pct * 100
all_non_pit_bull_return_per_capita

In [ ]:
breed = ["Non-Pit Bull", "Pit Bull or Pit Bull Mix"]
return_rate = [all_non_pit_bull_return_per_capita, all_pit_bull_return_per_capita]
x_axis = [1,2]

plt.bar(x_axis, return_rate, color='#4283f4', align="center")

tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, breed)

barlist=plt.bar([1], [all_non_pit_bull_return_per_capita])
barlist[0].set_color('#f49404')

plt.title("Outcomes Per 100 Adopted Animals", fontweight='bold')
plt.xlabel("Breeds", fontweight='bold')
plt.ylabel("Number of Returns", fontweight='bold')

plt.legend((p1[0], p2[0]), ('Unnamed', 'Named'))

plt.grid(axis='y', alpha=.20)
plt.savefig("Outcomes Pitbull vs. Non-Pitbull.png", dpi=1020)
plt.show()

In [ ]:
# set width of bar
barWidth = 0.25
 
# set height of bar
per_capita_non_pit_bull = [all_non_pit_bull_return_per_capita]
per_capita_pit_bull = [all_pit_bull_return_per_capita]
#return_to_owner_chart = [no_name_per_capita_return_to_owner, name_per_capita_return_to_owner]
 
# Set position of bar on X axis
r1 = np.arange(len(all_non_pit_bull_return_per_capita))
r2 = [x + barWidth for x in r1]
#r3 = [x + barWidth for x in r2]
 
# Make the plot
plt.bar(r1, per_capita_non_pit_bull, color='#217fd1', width=barWidth, edgecolor='white', label='Transfered')
plt.bar(r2, per_capita_pit_bull, color='#ed9b04', width=barWidth, edgecolor='white', label='Adopted')
#plt.bar(r3, return_to_owner_chart, color='#21d1b3', width=barWidth, edgecolor='white', label='Returned To Owner')

#21d1b3

# Add xticks on the middle of the group bars
plt.xlabel('Outcomes', fontweight='bold')
plt.xticks([r + barWidth for r in range(len(per_capita_non_pit_bull))], ['Owner Surrender'])
plt.ylabel('Number of Occurances', fontweight='bold')
plt.title('Outcomes Per 100 Animals', fontweight='bold')
# Create legend & Show graphic
plt.legend()
plt.grid(axis='y', alpha=.20)
plt.savefig("Outcomes Pitbull vs. Non-Pitbull.png", dpi=1020)
plt.show()